In [1]:
from pathlib import Path, PosixPath
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import shutil

### Gestione path

In [2]:
main_folder_path = Path('../../instagram-dataset')

In [3]:
path = main_folder_path.joinpath('dataset')

In [4]:
images = os.listdir(path=path)

In [5]:
main_formatted_path = main_folder_path.joinpath('dataset-formatted')
main_formatted_path.mkdir(parents=True, exist_ok=True)

In [6]:
dt_inf_list = pd.read_csv('../dataset/influencer_list.csv')
print("Numero di esempi: ", len(dt_inf_list))
dt_inf_list.head()

Numero di esempi:  1000


,Unnamed: 0,name,topic
0,0,cristiano,Soccer
1,1,kyliejenner,Modeling
2,2,selenagomez,Actors
3,3,leomessi,Soccer
4,4,beyonce,Singer


In [7]:
dt_inf_list = dt_inf_list[dt_inf_list['topic'].notna()]

In [8]:
dt_inf_list['topic'].value_counts()

Actors         251
Singer         104
Music           84
Modeling        83
Fashion         60
              ... 
Comedy           1
Apparel          1
Band             1
Comics           1
Journalists      1
Name: topic, Length: 65, dtype: int64

In [9]:
len(dt_inf_list['topic'].unique())

65

### Accorpamento categorie

In [10]:
mask_sport = dt_inf_list.topic.str.contains("Journalists|Coaching|Mixed martial arts|Riders|Cricket|Sports|Surfing|Soccer|American football|Tennis|Boxing|Basketball|Gym|Fitness|Training")
dt_inf_list.loc[mask_sport, 'topic'] = 'sport'

In [11]:
mask_music = dt_inf_list.topic.str.contains("Rap|Music|Band|DJ|Music of Asia|Songwriting|Singer|Producers")
dt_inf_list.loc[mask_music, 'topic'] = 'music'

In [12]:
mask_actor = dt_inf_list.topic.str.contains("Actors|Film|Movies")
dt_inf_list.loc[mask_actor, 'topic'] = 'actor'

In [13]:
mask_fashion = dt_inf_list.topic.str.contains("Photography|Design|Illustrator|Interior Design|Art|Beauty|Celebrities|Events|Fashion|Apparel|Modeling|Beauty|Streetwear|Styling")
dt_inf_list.loc[mask_fashion, 'topic'] = 'fashion'

In [14]:
mask_art = dt_inf_list.topic.str.contains("Editing|Politics|Medical Center|Chef|Anime|Entrepreneur|Business|Moms|Dance|Video Games|Humor|Lifestyle|Video Blogger|Entertainment|TV Host|Travel")
dt_inf_list.loc[mask_art, 'topic'] = 'entertainment' #influencer

In [15]:
dt_inf_list.head()

,Unnamed: 0,name,topic
0,0,cristiano,sport
1,1,kyliejenner,fashion
2,2,selenagomez,actor
3,3,leomessi,sport
4,4,beyonce,music


In [16]:
dt_inf_list = dt_inf_list.drop(
    dt_inf_list[
        dt_inf_list.topic.isin(['Cars','News&amp;Politics','Food', 'Comics','Memes','Children','Comedy','Interior Design','Xbox','TV Shows','TV Channel'])
    ].index
)

In [17]:
dt_inf_list['topic'].value_counts()

actor            262
music            222
fashion          185
entertainment    149
sport            144
Name: topic, dtype: int64

### Costruzione cartelle

Categorie eliminate:

- Xbox
- TV Shows
- TV Channel
- ...

In [18]:
len(dt_inf_list['topic'].unique())

5

In [19]:
for elem in dt_inf_list['topic'].unique():
    if str(elem) == elem:
        account = dt_inf_list.loc[dt_inf_list['topic'] == elem]
        main_formatted_path.joinpath(str(elem)).mkdir(parents=True, exist_ok=True)

        for acc in account['name']:
            for i in range(12):
                try:
                    acc_path = acc + str(i) + '.jpg'
                    my_file = path.joinpath(acc_path)
                    to_file = main_formatted_path.joinpath(elem)
                    shutil.copy(str(my_file), str(to_file))
                except OSError as e:
                    pass